In [1]:
################# Install compatible versions of torch and torchtext
!pip uninstall -y torch torchtext # uninstall previous verions to avoid conflits
!pip install torch==2.0.1
!pip install torchtext==0.15.2

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [2]:
################# ################# #################
################# Import packages
################# ################# #################
import numpy as np
import torchtext
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.manifold import MDS
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

OSError: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchtext/lib/libtorchtext.so, 0x0006): Symbol not found: __ZN3c105ErrorC1ENSt3__112basic_stringIcNS1_11char_traitsIcEENS1_9allocatorIcEEEES7_PKv
  Referenced from: <8349B302-A1C9-3870-AB5A-21A14A352BC2> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchtext/lib/libtorchtext.so
  Expected in:     <BA9C42A5-EA1D-3784-80E1-73FBFDE05847> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/lib/libc10.dylib

In [ ]:
################# ################# #################
################# Download glove
################# ################# #################
glove = torchtext.vocab.GloVe(name="6B", dim=100)


In [ ]:
A = glove['lamp']
B = glove['candle']

In [ ]:
################# ################# #################
################# Word representation
################# ################# #################
# Look at the representation of the word 'lamp', 'candle', their euclidian distance and cosine similarity
A = glove['lamp']
B = glove['candle']
euclidian  = np.sqrt(np.sum([(A[i]-B[i])**2 for i in range(np.shape(A)[0])])) # Using torch function euclidian = torch.norm(A - B)
cosine = np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B)) # cosine = torch.cosine_similarity(A.unsqueeze(0), B.unsqueeze(0))
print(euclidian, cosine)

In [ ]:
# Redo the same for mouse and lamp
A = glove['lamp']
B = glove['mouse']
euclidian  = np.sqrt(np.sum([(A[i]-B[i])**2 for i in range(np.shape(A)[0])]))
cosine = np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))
print(euclidian, cosine)

In [ ]:
# using cosine similarity, find the 10 closest words to the word lamp
A = glove['lamp']
allwords = np.array(glove.itos)
Similarity = []
for word in allwords:
    B = glove[word]
    cosine = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    Similarity.append(cosine)

max_index = np.argsort(Similarity)[-10:-1]
allwords[max_index]
print(allwords[max_index])


In [ ]:
# Write a function that takes as arg the embedding of a word and N and return the N closest words.
def returnclosest(word, n):
    allwords = np.array(glove.itos)
    Similarity = []
    for w in allwords:
        other_word = glove[w]
        cosine = np.dot(word, other_word) / (np.linalg.norm(word) * np.linalg.norm(other_word))
        Similarity.append(cosine)

    max_index = np.argsort(Similarity)[-(n+1):-1]
    return(allwords[max_index])


In [ ]:
close = returnclosest(glove['lamp'], 10)
print(close)

In [ ]:
close = returnclosest(glove['rabbit'], 10)
print(close)

In [ ]:
close = returnclosest(glove['bottle'], 10)
print(close)

In [ ]:
# Now that words are vectors, we can move into that space. For example find the embedding of 'queen' - 'woman' + 'man'
emb = glove['queen'] - glove['woman'] + glove['man']
# Find the closest word to this new embeding
returnclosest(emb, 1)

In [ ]:
# Check that the other realtion is true = 'king' - 'man' + 'woman' -> 'queen'
returnclosest(glove['king'] - glove['man'] + glove['woman'], 1)

In [ ]:
# Now let's investigate biaises in glove embedings : look at the 3 closest words to : doctor - man + woman
returnclosest(glove['doctor'] - glove['man'] + glove['woman'], 1)


In [ ]:
returnclosest(glove['doctor'] - glove['woman'] + glove['man'], 1)

In [ ]:

################# ################# #################
################# Dimensionality reduction
################# ################# #################
###### Now let's consider the 20 following words = ['rabbit', 'mouse', 'horse', 'bear', 'chimpanzee','whale, 'dophin', 'parrot', 'sparrow', 'jay', 'sofa', 'table', 'chair', 'rug', 'lamp', 'computer, 'phone', 'keyborard', 'screen', 'plate']
words = ['rabbit', 'mouse', 'horse', 'bear', 'monkey','whale', 'dolphin', 'tuna', 'swordfish', 'wolf', 'sofa', 'table', 'chair', 'rug', 'lamp','bag','computer', 'phone', 'keyboard', 'screen']
sim = np.zeros([np.shape(words)[0],np.shape(words)[0]])
for w1 in range(np.shape(words)[0]):
    W1 = glove[words[w1]]
    for w2 in range(np.shape(words)[0]):
        W2 = glove[words[w2]]
        sim[w1,w2] = np.dot(W1, W2) / (np.linalg.norm(W1) * np.linalg.norm(W2))


In [ ]:
plt.matshow(sim)
sim = pd.DataFrame(sim)
sim = sim.rename(mapper=pd.Series(words),axis=1)
ax = sns.clustermap(sim, cmap='coolwarm')

In [ ]:
#perform multi-dimensional scaling
mds = MDS(random_state=0)
scaled_df = mds.fit_transform(sim)
for i in range(np.shape(words)[0]):
    plt.scatter(scaled_df[:,0][i], scaled_df[:,1][i])
    plt.annotate(words[i], (scaled_df[:,0][i], scaled_df[:,1][i]))


In [ ]:
# perform PCA with one component
pca = PCA(n_components=1)
fig = plt.figure()
ax = fig.add_subplot(111)
X_reduced = pca.fit_transform(sim)
for i in range(np.shape(words)[0]):
    ax.scatter(X_reduced[:,0][i], 1)
    ax.text(X_reduced[:, 0][i], 1, words[i], rotation = 35)


In [ ]:
# perform PCA with two component
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(sim)
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(np.shape(words)[0]):
    ax.scatter(X_reduced[:,0][i], X_reduced[:,1][i])
    ax.text(X_reduced[:, 0][i], X_reduced[:, 1][i], words[i])

In [ ]:

# perform PCA with three component
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(sim)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
for i in range(np.shape(words)[0]):
    ax.scatter(X_reduced[:,0][i], X_reduced[:,1][i], X_reduced[:,2][i])
    ax.text(X_reduced[:,0][i], X_reduced[:,1][i], X_reduced[:,2][i],words[i])

In [ ]:
# Redo the sae with 800 first words.
words = glove.itos[0:800]
sim = np.zeros([np.shape(words)[0],np.shape(words)[0]])
for w1 in range(np.shape(words)[0]):
    W1 = glove[words[w1]]
    for w2 in range(np.shape(words)[0]):
        W2 = glove[words[w2]]
        sim[w1,w2] = np.dot(W1, W2) / (np.linalg.norm(W1) * np.linalg.norm(W2))

In [ ]:
sim = pd.DataFrame(sim)
sim = sim.rename(mapper=pd.Series(words),axis=1)
ax = sns.clustermap(sim, cmap='coolwarm')

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(sim)
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(np.shape(words)[0]):
    ax.scatter(X_reduced[:,0][i], X_reduced[:,1][i])
    ax.text(X_reduced[:, 0][i], X_reduced[:, 1][i], words[i], fontsize = 'xx-small')

In [ ]:
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(sim)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
for i in range(np.shape(words)[0]):
    ax.scatter(X_reduced[:,0][i], X_reduced[:,1][i], X_reduced[:,2][i])
    ax.text(X_reduced[:,0][i], X_reduced[:,1][i], X_reduced[:,2][i],words[i], fontsize = 'xx-small')

In [ ]:
# Kmeans on the similrity matrix and plot PCA colored with kmeans.
kmeans = KMeans(n_clusters=6, random_state=0).fit(sim)
kmeans.labels_


In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(sim)
colors = sns.color_palette("husl", 6)
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(np.shape(words)[0]):
    ax.scatter(X_reduced[:,0][i], X_reduced[:,1][i], color=colors[kmeans.labels_[i]])
    ax.text(X_reduced[:,0][i], X_reduced[:,1][i],words[i], fontsize = 'xx-small')

In [ ]:
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(sim)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
for i in range(np.shape(words)[0]):
    ax.scatter(X_reduced[:,0][i], X_reduced[:,1][i], X_reduced[:,2][i],color=colors[kmeans.labels_[i]])
    ax.text(X_reduced[:,0][i], X_reduced[:,1][i], X_reduced[:,2][i],words[i], fontsize = 'xx-small')